# Henter API

Her henter vi inn API-et og lagrer det i en csv fil
Vi henter data for lufttemperatur, nedmørsmengde og vindhastighet over 45 år i Oslo på værstasjonen Oslo-Blindern

In [5]:
# Importerer biblioteker
import requests
import pandas as pd

# Skriver inn client Id
client_id = '61f26ba4-3f68-4e56-a3b8-09733ceb82ed'

# Definerer endpoint og parametere
endpoint = 'https://frost.met.no/observations/v0.jsonld'
parameters = {
    'sources': 'SN18700',
    'elements': 'mean(air_temperature P1D),sum(precipitation_amount P1D),mean(wind_speed P1D)',
    'referencetime': '1975-01-01/2020-12-31',
}
# Sender request
r = requests.get(endpoint, parameters, auth=(client_id,''))
# Får inn json data
json = r.json()

# Sjekker requesten, om den er 200 er det godkjent og dataen er mottatt. Sjekker errorene hvis status_code ikke er 
if r.status_code == 200:
    data = json['data']
    print('Data retrieved from frost.met.no!')
else:
    print('Error! Returned status code %s' % r.status_code)
    print('Message: %s' % json['error']['message'])
    print('Reason: %s' % json['error']['reason'])


Data retrieved from frost.met.no!


# Vis data

Her viser vi at dataen eksisterer 

In [6]:
# Viser dataen i en dataframe som en tabell
df = pd.DataFrame()
for i in range(len(data)):
    row = pd.DataFrame(data[i]['observations'])
    row['referenceTime'] = data[i]['referenceTime']
    row['sourceId'] = data[i]['sourceId']
    df = pd.concat([df, row])

df = df.reset_index()

print(df)

df.to_csv('../data/data_45years.csv')

       index                      elementId  value  unit  \
0          0      mean(air_temperature P1D)   -2.3  degC   
1          1      mean(air_temperature P1D)   -0.4  degC   
2          2  sum(precipitation_amount P1D)    0.0    mm   
3          3  sum(precipitation_amount P1D)    7.4    mm   
4          4           mean(wind_speed P1D)    1.0   m/s   
...      ...                            ...    ...   ...   
83989      0      mean(air_temperature P1D)    1.7  degC   
83990      1      mean(air_temperature P1D)    2.3  degC   
83991      2  sum(precipitation_amount P1D)    5.9    mm   
83992      3  sum(precipitation_amount P1D)    5.6    mm   
83993      4           mean(wind_speed P1D)    5.1   m/s   

                                                   level timeOffset  \
0      {'levelType': 'height_above_ground', 'unit': '...       PT0H   
1      {'levelType': 'height_above_ground', 'unit': '...       PT6H   
2                                                    NaN      PT18

# Analyserer data

Vi ser at vi har mange unødvendige kolonner, som gjør dataen uoversiktlig. Derfor fjerner vi kolonnene: 
level, timeResolution, timeSeriesId, performanceCategory, exposureCategory, qualityCode, sourceId

Deretter lagrer vi dette i en ny csv-fil som vi kaller data_45years_update1

In [7]:
# Fjerner unødvendige kolonner
df = df.drop(['level','timeResolution','timeSeriesId','performanceCategory','exposureCategory','qualityCode','sourceId'], axis=1)
print(df)


df.to_csv('../data/data_45years_update1.csv')



       index                      elementId  value  unit timeOffset  \
0          0      mean(air_temperature P1D)   -2.3  degC       PT0H   
1          1      mean(air_temperature P1D)   -0.4  degC       PT6H   
2          2  sum(precipitation_amount P1D)    0.0    mm      PT18H   
3          3  sum(precipitation_amount P1D)    7.4    mm       PT6H   
4          4           mean(wind_speed P1D)    1.0   m/s       PT0H   
...      ...                            ...    ...   ...        ...   
83989      0      mean(air_temperature P1D)    1.7  degC       PT0H   
83990      1      mean(air_temperature P1D)    2.3  degC       PT6H   
83991      2  sum(precipitation_amount P1D)    5.9    mm      PT18H   
83992      3  sum(precipitation_amount P1D)    5.6    mm       PT6H   
83993      4           mean(wind_speed P1D)    5.1   m/s       PT0H   

                  referenceTime  
0      1975-01-01T00:00:00.000Z  
1      1975-01-01T00:00:00.000Z  
2      1975-01-01T00:00:00.000Z  
3      1975

# Legger til en kolonne 'dato'

Vi legger til en kolonne som viser dato, uten klokkeslett

In [8]:
#oppretter en ny kolone med dato uten tiden

df['date'] = pd.to_datetime(df['referenceTime']).dt.date

cols = ['date'] + [col for col in df.columns if col != 'date']
df = df[cols]

df.to_csv('../data/test_date.csv')

# Fjerner verdier som ikke gir mening
Vi velger å fjerne rader for temperatur der verdiene enten er under -50 eller over 60, fordi dette gir ikke mening og er sannsynligvis feil måling

In [20]:
# Bruk df.loc for å filtrere basert på 'elementId' og sjekk temperaturverdiene
df_filtered = df.loc[(df['elementId'] == 'mean(air_temperature P1D)') & 
                     (df['value'] >= -50) & 
                     (df['value'] <= 60)]

# Skriv ut den filtrerte DataFrame-en
print(df_filtered)

#må få det over i csv fil og sjekke om det var noen som ble fjernet

             date  index                  elementId  value  unit timeOffset  \
0      1975-01-01      0  mean(air_temperature P1D)   -2.3  degC       PT0H   
1      1975-01-01      1  mean(air_temperature P1D)   -0.4  degC       PT6H   
5      1975-01-02      0  mean(air_temperature P1D)   -0.2  degC       PT0H   
6      1975-01-02      1  mean(air_temperature P1D)   -1.3  degC       PT6H   
10     1975-01-03      0  mean(air_temperature P1D)    2.2  degC       PT0H   
...           ...    ...                        ...    ...   ...        ...   
83980  2020-12-28      1  mean(air_temperature P1D)    1.9  degC       PT6H   
83984  2020-12-29      0  mean(air_temperature P1D)    2.3  degC       PT0H   
83985  2020-12-29      1  mean(air_temperature P1D)    1.6  degC       PT6H   
83989  2020-12-30      0  mean(air_temperature P1D)    1.7  degC       PT0H   
83990  2020-12-30      1  mean(air_temperature P1D)    2.3  degC       PT6H   

                  referenceTime  
0      1975-01-01

# Finner gjennomsnittsverdier per dag
Vi ønsker å hente gjennomsnittet av målinger av de ulike elementene for hver dag, men ser at i filen er det for air_temperature og presipitation_amount to verdier per dag. Derfor vil vi finne gjennomsnittet av de elementene der vi har to verdier per dag. Med funksjonen pandas funksjon groupby fjernes også automatisk kolonnene timeOffset og referenceTime, slik vi ønsker.

In [9]:
# Gruppér og ta gjennomsnitt per dag og elementId
aggregert = df.groupby(['date', 'elementId']).agg({
    'value': 'mean',
    'unit': 'first',  # beholder enheten
}).round(3).reset_index()

aggregert.to_csv('../data/gjsnitt_data.csv')

print(aggregert)




             date                      elementId  value  unit
0      1975-01-01      mean(air_temperature P1D)  -1.35  degC
1      1975-01-01           mean(wind_speed P1D)   1.00   m/s
2      1975-01-01  sum(precipitation_amount P1D)   3.70    mm
3      1975-01-02      mean(air_temperature P1D)  -0.75  degC
4      1975-01-02           mean(wind_speed P1D)   1.00   m/s
...           ...                            ...    ...   ...
50392  2020-12-29           mean(wind_speed P1D)   5.30   m/s
50393  2020-12-29  sum(precipitation_amount P1D)   5.80    mm
50394  2020-12-30      mean(air_temperature P1D)   2.00  degC
50395  2020-12-30           mean(wind_speed P1D)   5.10   m/s
50396  2020-12-30  sum(precipitation_amount P1D)   5.75    mm

[50397 rows x 4 columns]
